# Project Title

## Introduction

Background:  With the improvement of society, people care more about their personal health and are willing to spend more money on health insurance. Medical costs may be affected by factors such as age, sex, and so on. Out of interest in this topic, our group selected this data set to make an analysis.

Research Question:  we are interested in what would be our best model in order to accurately predict the medical costs.

Data Overview: The data set contains one response variable, “charges”; and six explanatory variables, which are “age”, “sex”, “bmi”(Body mass index), “children”, “smoker”, and “region”, repectively. “Age”, “bmi” and “children” are continuous variables, and the other three variables are categorical. The factor “sex” and “smoker” have two levels, and the factor “region” has 4 levels. 
A linear regression model will be fitted to the dataset and we are going to explore how the different variables affect the model. Model selection methods will be applied in order to find the best model to make predictions.

## Preliminary Results

### Importing Modules

In [1]:
library(broom)
library(latex2exp)
library(tidyverse)
library(repr)
library(digest)
library(gridExtra)
library(faraway)
library(mltools)
library(leaps)
library(glmnet)
library(cowplot)
library(GGally)
library(infer)
library(cowplot)
library(AER)
library(math

ERROR: Error in parse(text = x, srcfile = src): <text>:17:0: unexpected end of input
15: library(AER)
16: library(math
   ^


### Downloading Dataset

In [ ]:
insurance <- read_csv("https://raw.githubusercontent.com/LukeXboy/STAT-301-Group-Project/main/insurance.csv")
head(insurance)

*Figure 1.1: Preview of Orginial Insurance Dataset*

### Feature Descriptions and Wrangling the Dataset

We look into the features of the dataset to see if we can wrangle the raw data into more comprehensable and useable forms.

The dataset contains the following variables relating to the patient:
- age
- sex
- bmi: Represents the Body Mass Index
- children: Number of children or dependents covered by the health insurance
- smoker: If the patient is a smoker
- region: Beneficiary's residentual area in the United States of America
- charges: Individual medical costs billed by the health insurance company

The features age, sex, bmi, children, smoker, and region are independent features or input variables in the dataset whereas charges is a dependent feature or a response variable. 

In [ ]:
#Produces further information about the feature values within the dataset
summary(insurance)

The output of the summary state us that age, bmi, children, and charges are continous variables in the dataset, and that there remaining variables, sex, smoker, and region are variables of type character. However, Figure 1.1 suggests that some of the variables in the dataset could be better represented as categorical variables for the purposes of our proposal; thus, we further analyze the feature values for the variables sex, children, smoker, and region.

In [ ]:
#Mutating the features to be categorical variables for further analysis
insurance_factor <- insurance %>%
    mutate(sex = as.factor(sex)) %>%
    mutate(children = as.factor(children)) %>%
    mutate(smoker = as.factor(smoker)) %>%
    mutate(region = as.factor(region)) 

head(insurance_factor)

#To check the feature values after mutating the variables
summary(insurance_factor)

*Figure 1.2 (top): Preview of Insurance Dataset with Mutated Categorical Variables*

After further analyzing the unique feature values for the variables sex, children, smoker, and region, it is clear that there are discrete values or levels for these variables. We confirm our speculations and implement the changes of wrangling the data set by mutating the variables to be categorical.

In [ ]:
#Implementing changes
insurance <- insurance_factor

#Printing number of rows related to the database
cat("Number of instances in the insurance dataset: ", nrow(insurance), "\n")
cat("Number of instances with missing values in the dataset: ", sum(is.na(insurance)), "\n")

Our dataset is now wrangled to accurately represent our data and contains no missing values, and ready to be split into the training and testing sets.

### Splitting Data into Training and Testing Sets

In [ ]:
#Splitting the Data by Labelling Dataset by ID
set.seed(123456)
insurance$ID <- rownames(insurance)
insurance_training <- sample_n(insurance, size = nrow(insurance) * 0.70, replace = FALSE)
insurance_testing <- anti_join(insurance, insurance_training, by = "ID")

insurance_training <- select(insurance_training, -ID)
insurance_testing <- select(insurance_testing, -ID)

head(insurance_training)
head(insurance_testing)

*Figure 1.3 (top): Preview of the training set of the dataset* 

*Figure 1.3 (bottom): Preview of the testing set of the dataset*

In [ ]:
#Printing number of rows within the split dataset
cat("Number of instances in the training set: ", nrow(insurance_training), "\n")
cat("Number of instances in the testing set: ", nrow(insurance_testing), "\n")

We split the dataset into a training and testing set as our project proposes to produce a predictive model which predicts charges on the new observations of patients. 

### Variable Distributions and Correlations

In [ ]:
#Histogram to Visualize Training Set Response Variable's, Charges, Distribution
options(repr.plot.width = 10, repr.plot.height = 8)
charges_hist <- insurance_training %>%
    ggplot(aes(x = charges)) + 
    geom_histogram(bins = 25, boundary = 0, colour = 'white') +
    labs(title = "Training Set Distribution of Patient's Medical Costs", 
         x = "Charges (USD)") +
    theme(text = element_text(size = 16)) +
    geom_vline(xintercept = mean(insurance$charges), colour = 'red', size = 3) 
charges_hist

*Figure 1.4: Histogram Displaying Training Set Distribution of Response Variable within Training , Charges, which represents the Medical Costs of the Patient. The red vertical line indicates the mean.*

In [ ]:
#Comparing Distribution of Input Variables to the Response Variable
options(repr.plot.width = 12, repr.plot.height = 8)
charges_dist_all <- insurance_training %>%
  ggduo(1:6, 7) +
  labs(title = "Distribution of Input Variables vs. Charges",
       x = "Input Variables",
       y = "Charges (USD)") +
  theme(
    text = element_text(size = 15),
    plot.title = element_text(face = "bold"),
    axis.title = element_text(face = "bold"),
    axis.text.x = element_text(angle = 45, vjust = 1, hjust=1) #To ensure labels do not overlap
  )
charges_dist_all

*Figure 1.4: Distribution of All Input Variables in the Training Dataset Against Charges, the Response Variable*

We analyze the distribution of our variables compared to the response variable, charges, and we notice that charges itself has a left skewed distribution. When comparing our response to our categorical variables, the level's of the variable smoker, which indicates if the patient smokes or not, showed the greatest variation across charges. The numerical variables showcase interesting trends in respective to the charges variable; however, we will look further into the correlation of the variables as a preview of whether or not our variables are independent and/or violate multicolinearity.

In [ ]:
#Computing Correlation Coefficients Across Continous Input Variables
cat("Correlation between the numeric type input variables, age and bmi: ", round(cor(insurance_training$age, insurance_training$bmi), 5))

Within the scope of the course, the correlation coefficient only applies to continous variables; thus we only analyze the correlations across the numeric type variables in our dataset. If provided the resources and ability to improve the course at a later point, correlations acorss non-numeric variables, such as Cramer's V and phi coefficient, can be researched and applied, and such information could possibly improve our model. 

The input variables age and bmi showcase a correlation of 0.10197 meaning the variables are weakly positively correlated which provides the assumption that there is no multicollinearity in the dataset. Therefore, the continous variables are not dependent on each other and thus would not decrease the precision of our estimated regression coefficients. During the project we will confirm this assumption through computing the Variance Inflation Factors for the input variables. 

## Preliminary Estimates 

In [ ]:
#Computing Expected Value of Response Variable
charges_mean <- mean(insurance_training$charges)
cat("Expected value of charges, E[Y]: ", charges_mean)

A baseline model is a simple model that acts as a reference, and within our project it is a model that predicts the response variable without any extra information. The baseline model's best prediction of the response is the sample mean of Y. Throughout our proposed project, we are to build a model which will predict or regress the response with some number of inputs that are to be selected through feature selection, which performs better than our better than nothing, intercept only, model. 

## Methods: Plan

As mentioned in the introduction part, we will apply some ideas of multiple linear regression and find the best model to predict the individual medical costs (health insurance) based on the data we have. Since we are building a predictive model, we might split the data set into a training set (for model construction) and testing set (for model prediction). We will use some parameters like R^2, adjusted R^2, Cp, BIC… to see if our model fits the data well and do some tests (ep. F-test) to decide what model we should use. We expect to build a MLR model which could accurately predict a person’s medical costs given some inputs (personal information). Hopefully, our projection results could help some medical organisations or even the government decide their financial assignment.



## References

Huang, J. Z. (2014). An introduction to statistical learning: With applications in R by gareth james, trevor hastie, robert tibshirani, daniela witten. Journal of Agricultural, Biological, and Environmental Statistics, 19(4), 419. https://doi.org/10.1007/s13253-014-0179-9

Lantz, B., & O'Reilly for Higher Education. (2015). Machine learning with R: Discover how to build machine learning algorithms, prepare data, and dig deep into data prediction techniques with R. Packt Publishing.
